In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
import pickle
pyLDAvis.enable_notebook()


In [2]:
root=os.getcwd()

In [3]:
data=pd.read_csv(root+'/dataset.csv', header=None, names=["Repo ID", "Language", "Text"])

In [4]:
repoList=np.unique(data["Repo ID"])

# 1 repo text

In [5]:
def trim(text):
    if type(text)==str:
        return text.lstrip().rstrip()
    elif type(text)==list:
        tram=[]
        for i in text:
            tram.append(i.lstrip().rstrip())
        return tram
    else:
        return "wutface"

In [6]:
def splat(text, splt):
    if type(text)==str:
        splot=str.split(text[2:-1], splt)
        return trim(splot)
    elif type(text)==list:
        splot=[]
        for i in text:
            for j in str.split(i,splt):
                splot.append(j)
        return trim(splot)
    else:
        return "wutface"

In [25]:
def scrub(string, inclComments=False):
    x=string
    x=splat(x,"\\n")
    if inclComments==False:
        x=[i for i in x if i[0:2]!="//"]
        x=[i for i in x if i[0:1]!="#"]
    x=splat(x,"(")
    x=splat(x,"[")
    x=splat(x,"{")
    x=splat(x," ")
    x=[i for i in x if i!=""]
    return x

In [35]:
def concat_texts(group):
    [repo_id] = group['Repo ID'].unique()
    combined = ' '.join(group['Text'])
    return combined

In [36]:
combined_documents = data.groupby(by='Repo ID').apply(concat_texts)
scrubbed_documents = combined_documents.apply(scrub)

In [44]:
" ".join(scrubbed_documents.iloc[0])

'NULL NULL " b\'#\\\' Calculated aesthetics stat <- function x) x } match_calculated_aes <- "^\\\\\\\\.\\\\\\\\. a-zA-Z._]+)\\\\\\\\.\\\\\\\\.$" is_dotted_var <- function x) grepl match_calculated_aes, x) } is_calculated_aes <- function aesthetics) vapply aesthetics, is_calculated, logical 1), USE.NAMES = FALSE) } is_calculated <- function x) if is.atomic x)) FALSE } else if is.symbol x)) is_dotted_var as.character x)) } else if is.call x)) if identical x 1]], quote stat))) TRUE } else any vapply x, is_calculated, logical 1))) } } else if is.pairlist x)) FALSE } else stop "Unknown input:", class x) 1]) } } strip_dots <- function expr) if is.atomic expr)) expr } else if is.name expr)) expr_ch <- as.character expr) if nchar expr_ch) > 0) as.name gsub match_calculated_aes, "\\\\\\\\1", expr_ch)) } else expr } } else if is.call expr)) if identical expr 1]], quote stat))) strip_dots expr 2]]) } else expr -1] <- lapply expr -1], strip_dots) expr } } else if is.pairlist expr)) as.pairlist lap

In [40]:
def concatMe(a):
    return " ".join(a)

In [41]:
superScrub=scrubbed_documents.apply(concatMe)

In [43]:
superScrub

Repo ID
19438        NULL NULL " b'#\' Calculated aesthetics stat <...
26554        import os from r2.lib.translation import I18N_...
544208       import logging import os import platform as py...
643909       r_env_vars <- function ) vars <- c "R_LIBS" = ...
2594513      process_group = function x) UseMethod \'proces...
3834332      \'\'\' \'\'\' from __future__ import print_fun...
4729944      shinyApp <- function ui=NULL, server=NULL, onS...
4751958      from .celery import app as celery_app __all__ ...
6427813      loc <- function data) .Call `_dplyr_loc`, data...
8162715      const ROOT = "../saleor/static/dashboard-next"...
10270250     'use strict'; const es5Paths, esNextPaths, } =...
13926404     from __future__ import print_function import s...
14098069     var gulp = require 'gulp'); var markdown = req...
14579179     add_data <- function p, data = NULL) if is.nul...
18840003     /** * React Starter Kit https://www.reactstart...
22003158     library rvest) library purrr) Link

In [45]:
tf_vectorizer = CountVectorizer(stop_words=None, token_pattern="[^ ]*")
tf = tf_vectorizer.fit_transform(superScrub)

In [46]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

with open(root+'/concatDocs_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [47]:
with open(root+'/concatDocs_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

In [49]:
vis=pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [53]:
pyLDAvis.show(vis)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [19/Feb/2019 12:43:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 12:43:08] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 12:43:08] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 12:43:08] code 404, message Not Found
127.0.0.1 - - [19/Feb/2019 12:43:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Feb/2019 12:43:08] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


## New Stuff

In [66]:
combinedAndReady=combined_documents

In [67]:
tf_vectorizer2 = CountVectorizer(stop_words=None)
tf2 = tf_vectorizer2.fit_transform(combinedAndReady)

In [68]:
number_of_languages = 4

lda2 = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model2 = lda2.fit(tf2)

with open(root+'/concatDocsNoScrub_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model2, f)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [69]:
with open(root+'/concatDocsNoScrub_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model2 = pickle.load(f)

In [70]:
vis2=pyLDAvis.sklearn.prepare(model2, tf2, tf_vectorizer2)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
pyLDAvis.show(vis2)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [19/Feb/2019 13:00:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 13:00:04] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 13:00:04] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2019 13:00:04] code 404, message Not Found
127.0.0.1 - - [19/Feb/2019 13:00:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Feb/2019 13:00:04] "GET /LDAvis.js HTTP/1.1" 200 -


In [65]:
combinedAndReady

Repo ID
19438        b"#' @include ggplot-global.R\n#' @include ggp...
26554        b'# The contents of this file are subject to t...
544208       b'import logging\nimport os\nimport platform a...
643909       b'#\' Environment variables to set when callin...
2594513      b'# S3 method to deal with chunks and inline t...
3834332      b'\'\'\'\n\n\'\'\'\nfrom __future__ import pri...
4729944      b'# TODO: Subapp global.R\n\n#\' Create a Shin...
4751958      b'from .celery import app as celery_app\n\n__a...
6427813      b'# Generated by using Rcpp::compileAttributes...
8162715      b'const ROOT = "../saleor/static/dashboard-nex...
10270250     b"'use strict';\n\nconst {\n  es5Paths,\n  esN...
13926404     b'#!/bin/env python\nfrom __future__ import pr...
14098069     b"var gulp = require('gulp');\nvar markdown = ...
14579179     b'#\' Add data to a plotly visualization\n#\' ...
18840003     b"/**\n * React Starter Kit (https://www.react...
22003158     b'library(rvest)\nlibrary(purrr)\n